

<p style="background-color:romance; color:jaguar; font-size:200%; text-align:center; border-radius:10px 10px; font-family:newtimeroman; line-height: 1.4; text-transform: title;"><strong>Credit Score Multi-class Classification</strong></p>

<p style="background-color:romance; color:jaguar; font-size:200%; text-align:center; border-radius:10px 10px; font-family:newtimeroman; line-height: 1.1;">Project Solution Models: Logistic Regression, Random Forest, XGBoost</p>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

# The style parameters control properties like the color of the background and whether a grid is enabled by default.
sns.set_style("whitegrid", {'axes.grid' : False})
# sns.set_style("whitegrid")

# Environment settings: 
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

In [3]:
import warnings
# Suppressing a warning 
warnings.filterwarnings("ignore") 
warnings.warn("this will not show")

In [4]:
# Statistics functions
from scipy import stats

In [5]:
df_origin_train = pd.read_csv('../input/credit-score-classification/train.csv')
df_train = df_origin_train.copy()

df_origin_test = pd.read_csv('../input/credit-score-classification/test.csv')
df_test = df_origin_test.copy()

df_train.shape, df_test.shape

FileNotFoundError: [Errno 2] No such file or directory: '../input/credit-score-classification/train.csv'

## Recognizing and Understanding Data

In [ ]:
(df_train.columns[:-1]!=df_test.columns).sum()

In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
df = df.applymap(lambda x: x if x is np.NaN or not isinstance(x, str) else str(x).strip('_ ,"')
                      ).replace(' and', '').replace(['', 'nan', '!@9#%8', '#F%$D@*&8'], np.NaN)
df

# 02. EDA
- Drop unnecessary colums

In [ ]:
df.select_dtypes(include='O').nunique()

In [ ]:
df.groupby('Month')['Customer_ID'].count().to_frame('count_Customer_ID').plot(kind ="bar")

In [ ]:
# Drop the Columns we not use in model
df.drop(columns=['ID', 'Month', 'Name', 'SSN'], inplace=True)

In [ ]:
df.select_dtypes(include='O').isna().sum()

## Fixing a data type (value_counts(), astype(), infer_objects(), convert_dtypes())

- Detect strange values by Columns Convert Object Types to Numeric Types (int, float, np.int64, pd.Int64Dtype())
- Combining object columns

Change column type in pandas:

  01. astype() - convert (almost) any type to (almost) any other type (even if it's not necessarily sensible to do so). Also   allows you to convert to categorial types (very useful).
  02. infer_objects() - a utility method to convert object columns holding Python objects to a pandas type if possible.
  03. convert_dtypes() - convert DataFrame columns to the "best possible" dtype that supports pd.NA (pandas' object to indicate a missing value).
  04. to_numeric() - provides functionality to safely convert non-numeric types.(See also to_datetime() and to_timedelta().)
  05. factorize() - provides sorting

### Assign Object Types to Numeric Types

In [ ]:
df.describe(include='O').T

In [ ]:
df['Customer_ID'] = df.Customer_ID.apply(lambda x: int(x[4:], 16))
df['Age'] = df.Age.astype(int)        
df['Annual_Income'] = df.Annual_Income.astype(float)
df['Num_of_Loan'] = df.Num_of_Loan.astype(int) 
df['Num_of_Delayed_Payment'] = df.Num_of_Delayed_Payment.astype(float)
df['Changed_Credit_Limit'] = df.Changed_Credit_Limit.astype(float)
df['Outstanding_Debt'] = df.Outstanding_Debt.astype(float)
df['Amount_invested_monthly'] = df.Amount_invested_monthly.astype(float)
df['Monthly_Balance'] = df.Monthly_Balance.astype(float)

#### Combining object columns

In [ ]:
def Month_Converter(x):
    if pd.notnull(x):
        num1 = int(x.split(' ')[0])
        num2 = int(x.split(' ')[3])
      
        return (num1*12)+num2
    else:
        return x
    
# Month_Converter('3 Years and 1 Months')

In [ ]:
df['Credit_History_Age'] = df.Credit_History_Age.apply(lambda x: Month_Converter(x)).astype(float)

In [ ]:
def get_Diff_Values_Colum(df_column, diff_value = []):   
    column = df_column.dropna()
    for i in column:
        if ',' not in i and i not in diff_value:
            diff_value.append(i)
        else:
            for data in map(lambda x:x.strip(), i.replace('and','').split(',')):
                if not data in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))

In [ ]:
get_Diff_Values_Colum(df['Type_of_Loan'])

## Detect Group NaN Values and Fill Group Mode

In [ ]:
df.info()

### Object Column: Get Group Mode and Reassign NaN Values

In [ ]:
# Reassign and Show Function
def Reassign_Group_Object_NaN_Values(df, groupby, column, inplace=True):      
    import scipy.stats as stats
    # Assigning Wrong values Make Simple Function
    def make_NaN_and_fill_mode(df, groupby, column, inplace=True):
        # Assign None to np.NaN
        if df[column].isin([None]).sum():
            df[column][df[column].isin([None])] = np.NaN
            
        # fill with local mode
        result = df.groupby(groupby)[column].transform(lambda x: x.fillna(stats.mode(x)[0][0]))

        # inplace
        if inplace:
            df[column]=result
        else:
            return result
    
    # Run and show some info     
    if inplace:  
        # Before Assigning Wrong values      
        x = df[column].value_counts(dropna=False).head(1)
        print(f'\nBefore Assigning NaN {column}:', [x if x.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        a = df.groupby(groupby)[column].apply(list) 
        print(f'\nBefore Assigning Example {column}:\n', *a.head().values, sep='\n', end='\n')
        
        # Assigning
        make_NaN_and_fill_mode(df, groupby, column, inplace)
        
        # After Assigning Wrong values
        y = df[column].value_counts(dropna=False).head(1)
        print(f'\nAfter Assigning NaN {column}:', [y if y.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        b = df.groupby(groupby)[column].apply(list)
        print(f'\nAfter Assigning Example {column}:\n', *b.head().values, sep='\n', end='\n')
    else:   
        # Show
        return make_NaN_and_fill_mode(df, groupby, column, inplace)

In [ ]:
df.describe(include='O').T

In [ ]:
Reassign_Group_Object_NaN_Values(df, 'Customer_ID', 'Occupation')
Reassign_Group_Object_NaN_Values(df, 'Customer_ID', 'Credit_Mix')
Reassign_Group_Object_NaN_Values(df, 'Customer_ID', 'Payment_Behaviour')

In [ ]:
df[df['Customer_ID']==1008]['Type_of_Loan']

In [ ]:
df['Type_of_Loan'] = df['Type_of_Loan'].replace([', and ', ', '], ',', regex=True)

In [ ]:
df['Type_of_Loan'] = df['Type_of_Loan'].fillna('Not Specified')

### Numeric Column: Get Group Min-Max Detect Outlier-Wrong V alues and Reassign NaN Values

In [ ]:
# Reassign Wrong Values and Show Function
def Reassign_Group_Numeric_Wrong_Values(df, groupby, column, inplace=True):      
    import scipy.stats as stats 

    # Identify Wrong values Range
    def get_group_min_max(df, groupby, column):            
        cur = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = cur.apply(lambda x: stats.mode(x)).apply([min, max])
        return x[0][0], y[0][0]
    
    # Assigning Wrong values
    def make_group_NaN_and_fill_mode(df, groupby, column, inplace=True):
        df_dropped = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = df_dropped.apply(lambda x: stats.mode(x)).apply([min, max])
        mini, maxi = x[0][0], y[0][0]

        # assign Wrong Values to NaN
        col = df[column].apply(lambda x: np.NaN if ((x<mini)|(x>maxi)) else x)

        # fill with local mode
        mode_by_group = df.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)
        result = col.fillna(mode_by_group)

        # inplace
        if inplace:
            df[column]=result
        else:
            return result
        
    
    # Run      
    if inplace:   
        print("\nExisting Min, Max Values:", df[column].apply([min, max]), sep='\n', end='\n')       
        mini, maxi = get_group_min_max(df, groupby, column)        
        print(f"\nGroupby by {groupby}'s Actual min, max Values:", f'min:\t{mini},\nmax:\t{ maxi}', sep='\n', end='\n')
        
        # Before Assigning Wrong values      
        x = df[column].value_counts(dropna=False).head(1)
        print(f'\nBefore Assigning Min, Max {column}:', [x if x.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        
        a = df.groupby(groupby)[column].apply(list) 
        print(f'\nBefore Assigning Example {column}:\n', *a.head().values, sep='\n', end='\n')
        
        # Assigning
        make_group_NaN_and_fill_mode(df, groupby, column, inplace)
        
        # After Assigning Wrong values
        y = df[column].value_counts(dropna=False).head(1)
        print(f'\nAfter Assigning  Min, Max {column}:', [y if y.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        
        b = df.groupby(groupby)[column].apply(list)
        print(f'\nAfter Assigning Example {column}:\n', *b.head().values, sep='\n', end='\n')
    else:   
        # Show
        return make_group_NaN_and_fill_mode(df, groupby, column, inplace)

In [ ]:
df.describe().T

In [ ]:
df.select_dtypes(include='number').isna().sum()

In [ ]:
# reassign Group nan and Wrong-outlier values
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Age')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Annual_Income')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Inhand_Salary')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Bank_Accounts')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Card')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Interest_Rate')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Loan')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Delay_from_due_date')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Delayed_Payment')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Changed_Credit_Limit')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Inquiries')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Outstanding_Debt')
df['Credit_Utilization_Ratio'] = df.groupby('Customer_ID')['Credit_Utilization_Ratio'].apply(lambda x: x.interpolate().bfill().ffill())
df['Credit_History_Age'] = df.groupby('Customer_ID')['Credit_History_Age'].apply(lambda x: x.interpolate().bfill().ffill())
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Total_EMI_per_month')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Amount_invested_monthly')
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Balance')

In [ ]:
df['Monthly_Balance'][df['Monthly_Balance']<0]

In [ ]:
df.loc[df['Monthly_Balance']<0, 'Monthly_Balance'] = None

In [ ]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Balance')

In [ ]:
df.isna().sum()

In [ ]:
# Drop the Columns we not use in model
df.drop(columns=['Customer_ID'], inplace=True)

In [ ]:
# Drop the Rows we can not use in model
df = df[df['Credit_Score'].notna()]
df.shape

In [ ]:
# Check Target Feature for Classification
fig, (hist, box) = plt.subplots(nrows=2, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
sns.histplot(df['Credit_Score'], discrete=True, stat='percent', shrink=.8, kde=False, ax=hist).grid(axis='y')
hist.bar_label(hist.containers[0], fmt='%% %.0f', padding=-21, color='w', fontsize=14);
sns.boxplot(pd.factorize(df['Credit_Score'])[0], ax=box, showmeans=True).grid(axis='x')

In [ ]:
# Correlation between Numerical features
plt.figure(figsize=(14,8))
sns.heatmap(
    df.corr(), 
    mask=np.triu(np.ones_like(df.corr(), dtype=bool)), 
    annot=True, vmin=-1, vmax=1, cmap="PiYG"
)

In [ ]:
# df.to_csv("clean_credit_score_classification.csv", index=False)

# 03. Multi-class Classification Data Pre-Processing

**Task:** Identify people with low credit scores

01. Import Libraries and Reading the Data from File
02. EDA
03. Multi-class Classification Data Pre-Processing
    01. Implement Logistic Regression Model
    02. Implement Random Forest Classifier Model
    03. Implement Xgboost Classifier Model
04. Future Importance XGBoost 
05. Final Model
06. Prepare Model Deployment

## Import Libraries

In [ ]:
# conda install -c anaconda scikit-learn
# sklearn library for machine learning algorithms, data preprocessing, and evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline

# Supervised-Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
# conda install -c conda-forge xgboost
from xgboost import XGBClassifier
# Supervised-Classifier-metrics
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, f1_score, log_loss
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix
from sklearn.metrics import plot_roc_curve, roc_curve, roc_auc_score, auc
from sklearn.metrics import plot_precision_recall_curve, precision_recall_curve, average_precision_score

# Supervised-cross_validate-GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import pickle
random_state=42

In [ ]:
df1 = df.copy()

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

## Train | Test Split

In [ ]:
X = df.drop(columns="Credit_Score")
y = df['Credit_Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

X_train.shape, X_test.shape, y_train.shape, 

## Dummy Operation

In [ ]:
X_train = X_train.join(X_train["Type_of_Loan"].str.get_dummies(sep = ",").add_prefix("tol_"))
X_test = X_test.join(X_test["Type_of_Loan"].str.get_dummies(sep = ",").add_prefix("tol_"))
X_test = X_test.reindex(columns = X_train.columns, fill_value=0) # "0"

X_train.drop(columns=["Type_of_Loan"], inplace = True)
X_test.drop(columns=["Type_of_Loan"], inplace = True)

X_train.shape, X_test.shape

## OneHotEncoder and LabelEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat = X_train.select_dtypes(exclude="number").columns.to_list()    
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)

X_train_cat = pd.DataFrame(
    enc.fit_transform(X_train[cat]), index = X_train.index, 
    columns = enc.get_feature_names_out(cat)
)    
X_test_cat  = pd.DataFrame(
    enc.transform(X_test[cat]), index = X_test.index, 
    columns = enc.get_feature_names_out(cat)
)    
X_train_enc = X_train_cat.join(X_train.select_dtypes("number"))
X_test_enc = X_test_cat.join(X_test.select_dtypes("number"))

X_train_enc.shape, X_test_enc.shape

In [ ]:
X_train_enc.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

pd.DataFrame(y_train).value_counts().sort_index()

## Scale data

In [ ]:
scaler = MinMaxScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_enc), columns=X_train_enc.columns)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test_enc), columns=X_test_enc.columns)

# 01. Implement Logistic Regression Model

In [ ]:
log_model = LogisticRegression(
    class_weight = "balanced",
    random_state=random_state
)

In [ ]:
log_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred_train = log_model.predict(X_train_scaled)
y_pred_test  = log_model.predict(X_test_scaled)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

## With Best Parameters (GridsearchCV)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model = LogisticRegression(    
    class_weight = "balanced",
    max_iter=10000,
    random_state=random_state
)
param_grid = {
    "penalty" : ['l1'],
    "C" : [0.1],
    "solver" : ['saga'], # 'liblinear', 
}
# Identify people with low credit scores
recall_poor = make_scorer(recall_score, average = None, labels=[1])
grid_model_log = GridSearchCV(
    estimator=model,
    param_grid = param_grid, 
    scoring=recall_poor, 
    error_score="raise",
    n_jobs=-1,
    cv=5
) 

In [ ]:
%%time
grid_model_log.fit(X_train_scaled, y_train)

In [ ]:
grid_model_log.best_params_

In [ ]:
grid_model_log.best_score_

In [ ]:
y_pred_train = grid_model_log.predict(X_train_scaled)
y_pred_test  = grid_model_log.predict(X_test_scaled)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

# 02. Implement Random Forest Classifier Model

## OrdinalEncoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

cat = X_train.select_dtypes(exclude="number").columns.to_list()    
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train[cat] = enc.fit_transform(X_train[cat])
X_test[cat] = enc.transform(X_test[cat])

X_train.shape, X_test.shape

## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(
    class_weight = 'balanced',
    random_state=random_state
)

In [ ]:
%%time
rfc_model.fit(X_train, y_train)

In [ ]:
y_pred_train = rfc_model.predict(X_train)
y_pred_test  = rfc_model.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

## With Best Parameters (GridsearchCV)

In [ ]:
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(
    class_weight = 'balanced',
    random_state=random_state
)
param_grid = {
    'max_depth': [3], 
    'n_estimators': [10],
    'max_features': [None],
}
# Identify people with low credit scores
recall_poor = make_scorer(recall_score, average = None, labels=[1])
grid_model_rfc = GridSearchCV(
    estimator=model,
    param_grid = param_grid, 
    error_score="raise",
    scoring=recall_poor,
    refit=True,
#     verbose=2,
    n_jobs=-1,
    cv=5,
)

In [ ]:
%%time
grid_model_rfc.fit(X_train, y_train)

In [ ]:
grid_model_rfc.best_params_

In [ ]:
grid_model_rfc.best_score_

In [ ]:
y_pred_train = grid_model_rfc.predict(X_train)
y_pred_test  = grid_model_rfc.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

# 03. Implement XGBoost Classifier Model

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    random_state=random_state,
)

In [ ]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
classes_weights

In [ ]:
comp = pd.DataFrame(classes_weights)

comp["label"] = pd.DataFrame(y_train).reset_index(drop=True)
comp.groupby("label")[0].value_counts()

In [ ]:
%%time
xgb_model.fit(
    X_train, y_train, 
    sample_weight=classes_weights
)
# weight parameter in XGBoost is per instance not per class.

In [ ]:
y_pred_train = xgb_model.predict(X_train)
y_pred_test  = xgb_model.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

## With Best Parameters (GridsearchCV)

In [ ]:
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(
    random_state=random_state
)
param_grid = {
    'n_estimators':[50, 100],
}
# Identify people with low credit scores
recall_poor = make_scorer(recall_score, average = None, labels=[1])
grid_model_xgb = GridSearchCV(
    estimator=model,
    param_grid = param_grid, 
    error_score="raise",
    scoring=recall_poor,
    refit=True,
    n_jobs=-1
)

In [ ]:
%%time
grid_model_xgb.fit(
    X_train, y_train, 
    sample_weight=classes_weights
)

In [ ]:
grid_model_xgb.best_params_

In [ ]:
grid_model_xgb.best_score_

In [ ]:
y_pred_train = grid_model_xgb.predict(X_train)
y_pred_test  = grid_model_xgb.predict(X_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

# Compare The Models

In [ ]:
from yellowbrick.classifier import ROCAUC
model = grid_model_log

visualizer = ROCAUC(model, encoder={0: 'Good', 1: 'Poor', 2: 'Standart'})
visualizer.fit(X_train_scaled, y_train)        # Fit the training data to the visualizer
visualizer.score(X_train_scaled, y_train)        # Evaluate the model on the test data
visualizer.show(); 

In [ ]:
from yellowbrick.classifier import ROCAUC
model = grid_model_rfc

visualizer = ROCAUC(model, encoder={0: 'Good', 1: 'Poor', 2: 'Standart'})
visualizer.fit(X_train, y_train)        # Fit the training data to the visualizer
visualizer.score(X_train, y_train)        # Evaluate the model on the test data
visualizer.show(); 

In [ ]:
from yellowbrick.classifier import ROCAUC
model = grid_model_xgb

visualizer = ROCAUC(model, encoder={0: 'Good', 1: 'Poor', 2: 'Standart'})
visualizer.fit(X_train, y_train)        # Fit the training data to the visualizer
visualizer.score(X_train, y_train)        # Evaluate the model on the test data
visualizer.show(); 

In [ ]:
plot_confusion_matrix(grid_model_rfc, X_test, y_test);
plot_confusion_matrix(grid_model_xgb, X_test, y_test);

# 04. Future Importance XGBoost 
- Permutation Based Feature Importance (with scikit-learn)

In [ ]:
# xgb_model has best scorer default parameters
xgb_model.feature_importances_

In [ ]:
plt.figure(figsize=(14,8))
plt.barh(X_test.columns, xgb_model.feature_importances_)

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(xgb_model, X_test, y_test)
perm_importance

In [ ]:
sorted_idx = perm_importance.importances_mean.argsort()
plt.figure(figsize=(14,8))
plt.barh(X_test.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

In [ ]:
X_test.columns

# 05. Final Model

In [ ]:
# we selected 8 features for final model
df_final = df[[
    'Credit_Mix', 'Interest_Rate', 'Outstanding_Debt', 'Delay_from_due_date', 
    'Total_EMI_per_month', 'Changed_Credit_Limit', 'Monthly_Inhand_Salary', 'Annual_Income',
    'Credit_Score'
]]
df_final

In [ ]:
X = df_final.drop(columns='Credit_Score')
y = df_final['Credit_Score']

In [ ]:
cat = X.select_dtypes(exclude="number").columns.to_list()    
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X[cat] = enc.fit_transform(X[cat])

X.shape

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y)
classes_weights

In [ ]:
final_model = XGBClassifier(
    random_state=random_state,
)

In [ ]:
%%time
final_model.fit(
    X, y, 
    sample_weight=classes_weights
)

In [ ]:
y_pred = final_model.predict(X)

print(classification_report(y, y_pred))

In [ ]:
mean_human = {
 'Credit_Mix': 'Standard',
 'Interest_Rate': 14,
 'Outstanding_Debt': 1426,
 'Delay_from_due_date': 21,
 'Total_EMI_per_month': 107,
 'Changed_Credit_Limit': 10,
 'Monthly_Inhand_Salary': 4197,
 'Annual_Income': 50505,
}
mean_human = pd.DataFrame([mean_human])
mean_human[cat] = enc.transform(mean_human[cat])
mean_human

In [ ]:
predict = final_model.predict(mean_human)
predict, le.inverse_transform(predict)

# 06. Prepare Model Deployment

In [ ]:
import pickle
pickle.dump(final_model, open("final_model_credit_score_pred", 'wb'))